# Scraping events

## Sito unpliveneto.it
### E' da capire come estrarre la DW-SessionID se scade

In [ ]:
import requests

url = "https://webapi.deskline.net/unpliveneto/it/events"
params = {
    "filterId": "",
    "fields": "id,name,dbCode,owner,isTopEvent,visibilityLevel,date,hasMoreDates,onlineBookable,location{place,town,regions,country,coordinate{name,long,lat}},plainDescriptions(len:50){description,type},descriptions(types:[32,33]){description,type},dateStartTimes,mainCriteria{id,name,value},criteria{groupId,groupName,items{id,name,value}},eventGroups{id,name},holidayThemes{id,name,order},images(count:1,sizes:[54]){id,name,extension,copyright,author,license,urls,resolutionX,resolutionY,description},urlFriendlyName,startTimeDurations{time,weekDays,duration,},guestCards{id,name,type,hasIcon,iconUrl,webLink}",
    "sortingFields": "date,-topEvent,time",
    "pageNo": 1,
    "pageSize": 1,
    "hashF": 0
}

headers = {
    "DW-Source": "desklineweb",
    "DW-SessionID": "G1758362087062",  # devi aggiornare dinamicamente da richiesta reale o ripetuta
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.unpliveneto.it/",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers, params=params)

print(response.status_code)
print(response.json())


In [ ]:
import requests
import pandas as pd

def fetch_unpli_events(page_no=1, page_size=24, session_id="G1758362087062"):
    url = "https://webapi.deskline.net/unpliveneto/it/events"
    params = {
        "filterId": "",
        "fields": (
            "id,name,dbCode,owner,isTopEvent,visibilityLevel,date,hasMoreDates,"
            "onlineBookable,location{place,town,regions,country,coordinate{name,long,lat}},"
            "plainDescriptions(len:50){description,type},descriptions(types:[32,33]){description,type},"
            "dateStartTimes,mainCriteria{id,name,value},criteria{groupId,groupName,items{id,name,value}},"
            "eventGroups{id,name},holidayThemes{id,name,order},images(count:1,sizes:[54]){id,name,extension,"
            "copyright,author,license,urls,resolutionX,resolutionY,description},urlFriendlyName,"
            "startTimeDurations{time,weekDays,duration,},guestCards{id,name,type,hasIcon,iconUrl,webLink}"
        ),
        "sortingFields": "date,-topEvent,time",
        "pageNo": page_no,
        "pageSize": page_size,
        "hashF": 0
    }

    headers = {
        "DW-Source": "desklineweb",
        "DW-SessionID": session_id,
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://www.unpliveneto.it/",
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print("Errore nella richiesta:", response.status_code)
        print(response.text)
        return None

    try:
        data = response.json()
        return data.get("data") or data.get("events")
    except Exception as e:
        print("Errore nel parsing JSON:", e)
        return None

def events_to_dataframe(events):
    rows = []
    for event in events:
        location = event.get("location", {})
        place = location.get("place", "")
        town = location.get("town", "")
        date = event.get("date", "")
        name = event.get("name", "")
        
        # Prendi la descrizione breve, se presente
        descs = event.get("plainDescriptions", [])
        description = ""
        if descs:
            description = descs[0].get("description", "")
        
        rows.append({
            "name": name,
            "place": place,
            "town": town,
            "date": date,
            "description": description,
            "startTimeDurations": event.get("startTimeDurations", []),
            "durations": [d.get("duration") for d in event.get("startTimeDurations", []) if isinstance(d, dict)]
        })
    return pd.DataFrame(rows)

# Esempio d'uso
session_id = "G1758362087062"  # Sostituisci con il valore effettivo
events = fetch_unpli_events(page_no=1, page_size=500, session_id=session_id)

if events:
    df = events_to_dataframe(events)
    try:
        from IPython.display import display
        display(df.head(500))
    except ImportError:
        print(df.head().to_string())
else:
    print("Nessun evento o errore nella risposta.")



In [ ]:
import requests
import pandas as pd

def fetch_unpli_events(page_no=1, page_size=24, session_id="G1758362087062"):
    url = "https://webapi.deskline.net/unpliveneto/it/events"
    params = {
        "filterId": "",
        "fields": (
            "id,name,dbCode,owner,isTopEvent,visibilityLevel,date,hasMoreDates,"
            "onlineBookable,location{place,town,regions,country,coordinate{name,long,lat}},"
            "plainDescriptions(len:50){description,type},descriptions(types:[32,33]){description,type},"
            "dateStartTimes,mainCriteria{id,name,value},criteria{groupId,groupName,items{id,name,value}},"
            "eventGroups{id,name},holidayThemes{id,name,order},images(count:1,sizes:[54]){id,name,extension,"
            "copyright,author,license,urls,resolutionX,resolutionY,description},urlFriendlyName,"
            "startTimeDurations{time,weekDays,duration,},guestCards{id,name,type,hasIcon,iconUrl,webLink}"
        ),
        "sortingFields": "date,-topEvent,time",
        "pageNo": page_no,
        "pageSize": page_size,
        "hashF": 0
    }

    headers = {
        "DW-Source": "desklineweb",
        "DW-SessionID": session_id,
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://www.unpliveneto.it/",
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print("Errore nella richiesta:", response.status_code)
        print(response.text)
        return None

    try:
        data = response.json()
        return data.get("data") or data.get("events")
    except Exception as e:
        print("Errore nel parsing JSON:", e)
        return None

def events_to_dataframe(events):
    rows = []
    for event in events:
        location = event.get("location") or {}
        coordinate = location.get("coordinate") or {}
        images = event.get("images") or []
        img_url = ""
        if images and isinstance(images, list):
            urls = images[0].get("urls", []) if images[0] else []
            if urls:
                img_url = urls[0]
        descriptions = event.get("descriptions") or []
        long_description = ""
        if descriptions and isinstance(descriptions, list):
            long_description = descriptions[0].get("description", "")

        start_time_durations = event.get("startTimeDurations") or []
        start_times = ", ".join([str(dur.get("time", "")) for dur in start_time_durations if isinstance(dur, dict)])

        rows.append({
            "id": event.get("id", ""),
            "name": event.get("name", ""),
            "dbCode": event.get("dbCode", ""),
            "owner": event.get("owner", ""),
            "isTopEvent": event.get("isTopEvent", False),
            "visibilityLevel": event.get("visibilityLevel", ""),
            "date": event.get("date", ""),
            "hasMoreDates": event.get("hasMoreDates", False),
            "onlineBookable": event.get("onlineBookable", False),
            "place": location.get("place", ""),
            "town": location.get("town", ""),
            "regions": ", ".join(location.get("regions", [])),
            "country": location.get("country", ""),
            "latitude": coordinate.get("lat"),
            "longitude": coordinate.get("long"),
            "description_short": event.get("plainDescriptions", [{}])[0].get("description", "") if event.get("plainDescriptions") else "",
            "description_long": long_description,
            "image_url": img_url,
            "urlFriendlyName": event.get("urlFriendlyName", ""),
            "start_times": start_times,
        })
    return pd.DataFrame(rows)


# Esempio d'uso
session_id = "G1758362087062"  # Sostituisci con il valore reale della sessione
events = fetch_unpli_events(page_no=1, page_size=2, session_id=session_id)

if events:
    df = events_to_dataframe(events)
    try:
        from IPython.display import display
        display(df)
    except ImportError:
        print(df.to_string())
else:
    print("Nessun evento o errore nella risposta.")


## Estraggo le date successive, se ce ne sono, così duplico gli eventi

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import json
import time


def clean_html(raw_html: str) -> str:
    if not raw_html:
        return ""
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text(separator=" ", strip=True)


def fetch_unpli_events(page_no=1, page_size=5, session_id="G1758362087062"):
    url = "https://webapi.deskline.net/unpliveneto/it/events"
    params = {
        "filterId": "",
        "fields": (
            "id,name,dbCode,owner,isTopEvent,visibilityLevel,date,hasMoreDates,"
            "onlineBookable,location{place,town,regions,country,coordinate{name,long,lat}},"
            "plainDescriptions(len:50){description,type},descriptions(types:[32,33]){description,type},"
            "dateStartTimes,mainCriteria{id,name,value},criteria{groupId,groupName,items{id,name,value}},"
            "eventGroups{id,name},holidayThemes{id,name,order},images(count:1,sizes:[54]){id,name,extension,"
            "copyright,author,license,urls,resolutionX,resolutionY,description},urlFriendlyName,"
            "startTimeDurations{time,weekDays,duration,},guestCards{id,name,type,hasIcon,iconUrl,webLink}"
        ),
        "sortingFields": "date,-topEvent,time",
        "pageNo": page_no,
        "pageSize": page_size,
        "hashF": 0
    }
    headers = {
        "DW-Source": "desklineweb",
        "DW-SessionID": session_id,
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://www.unpliveneto.it/",
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print("Errore nella richiesta:", response.status_code)
        print(response.text)
        return None
    try:
        data = response.json()
        return data.get("data") or data.get("events")
    except Exception as e:
        print("Errore nel parsing JSON:", e)
        return None


def fetch_event_details_dates(dbCode, event_id, session_id, from_date=None, max_retries=5):
    url = f"https://webapi.deskline.net/unpliveneto/it/events/{dbCode}/{event_id}"
    if from_date is None:
        from_date = "2020-01-01"
    fields_value = f'nextOccurrences(fromDate:"{from_date}",count:100){{items{{date,dayOfWeek,startTime,duration}},hasMoreItems}}'
    params = {"fields": fields_value}
    headers = {
        "DW-Source": "desklineweb",
        "DW-SessionID": session_id,
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://www.unpliveneto.it/",
        "User-Agent": "Mozilla/5.0"
    }
    backoff = 1
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, params=params)
            if response.status_code == 429:
                retry_after = response.headers.get("Retry-After")
                wait_time = int(retry_after) if retry_after and retry_after.isdigit() else backoff
                print(f"Received 429, retrying after {wait_time} seconds...")
                time.sleep(wait_time)
                backoff = min(backoff * 2, 60)
                continue
            response.raise_for_status()
            data = response.json()
            items = data.get("nextOccurrences", {}).get("items", [])
            dates_with_duration = []
            for item in items:
                if "date" in item:
                    dates_with_duration.append((f"{item['date'][:10]}T{item.get('startTime', '00:00')}:00", item.get("duration", 0)))
            time.sleep(1)  # Additional polite pause
            return dates_with_duration
        except requests.RequestException as e:
            print(f"Request error: {e}, attempt {attempt + 1} of {max_retries}")
            time.sleep(backoff)
            backoff = min(backoff * 2, 60)
    print("Max retries reached for event details, skipping dates.")
    return []


def transform_events_for_json(events, session_id):
    transformed = []
    total_events = len(events)
    for i, event in enumerate(events, 1):
        print(f"Processing event {i} of {total_events}: ID {event.get('id', '')}")
        descriptions = event.get("descriptions") or []
        long_description = ""
        if descriptions and isinstance(descriptions, list):
            raw_html = descriptions[0].get("description", "")
            long_description = clean_html(raw_html)

        location = event.get("location") or {}
        coordinate = location.get("coordinate") or {}

        venue = location.get("place", "")
        city = location.get("town", "")
        regions = location.get("regions") or []
        region_str = ", ".join(regions) if regions else ""

        title = event.get("name", "")

        category = ""
        criteria = event.get("criteria") or []
        if criteria and isinstance(criteria, list) and len(criteria) > 0:
            category = criteria[0].get("groupName", "") if criteria[0] else ""

        base_url = "https://www.unpliveneto.it/eventi-delle-pro-loco-in-veneto/#/eventi/"
        db_code = event.get("dbCode", "")
        event_id = event.get("id", "")
        url_friendly = event.get("urlFriendlyName", "")
        event_url = f"{base_url}{db_code}/{event_id}/{url_friendly}" if db_code and event_id and url_friendly else base_url

        if event.get("hasMoreDates", False):
            from_date = event.get("date")[:10]
            dates_with_durations = fetch_event_details_dates(dbCode=db_code, event_id=event_id, session_id=session_id, from_date=from_date)
            if not dates_with_durations:
                dates_with_durations = [(event.get("date"), 0)]
        else:
            dates_with_durations = [(event.get("date"), 0)]

        for start_date, duration_hours in dates_with_durations:
            try:
                dt_start = datetime.strptime(start_date[:19], "%Y-%m-%dT%H:%M:%S")
                if duration_hours == 0:
                    dt_end = dt_start.replace(hour=23, minute=59, second=59, microsecond=0)
                else:
                    dt_end = dt_start + timedelta(hours=duration_hours)
                end_date = dt_end.isoformat() + "Z"
            except Exception:
                end_date = start_date

            address = ", ".join([venue, city]) if venue and city else venue or city

            transformed.append({
                "id": event.get("id", ""),
                "title": title,
                "category": category,
                "description": long_description,
                "city": city,
                "location": {
                    "venue": venue,
                    "address": address,
                    "latitude": coordinate.get("lat"),
                    "longitude": coordinate.get("long")
                },
                "start_date": start_date,
                "end_date": end_date,
                "url": event_url
            })

    return transformed


def events_to_dataframe(events, session_id):
    rows = []
    for event in events:
        if event.get("hasMoreDates", False):
            from_date = event.get("date")[:10]
            dates = fetch_event_details_dates(event.get("dbCode", ""), event.get("id", ""), session_id, from_date=from_date)
            if not dates:
                dates = [event.get("date")]
        else:
            dates = [event.get("date")]

        location = event.get("location") or {}
        coordinate = location.get("coordinate") or {}
        images = event.get("images") or []
        img_url = ""
        if images and isinstance(images, list):
            urls = images[0].get("urls", []) if images[0] else []
            if urls:
                img_url = urls[0]
        descriptions = event.get("descriptions") or []
        long_description = ""
        if descriptions and isinstance(descriptions, list):
            raw_html = descriptions[0].get("description", "")
            long_description = clean_html(raw_html)

        start_time_durations = event.get("startTimeDurations") or []
        start_times = ", ".join([str(dur.get("time", "")) for dur in start_time_durations if isinstance(dur, dict)])

        for date in dates:
            rows.append({
                "id": event.get("id", ""),
                "name": event.get("name", ""),
                "dbCode": event.get("dbCode", ""),
                "owner": event.get("owner", ""),
                "isTopEvent": event.get("isTopEvent", False),
                "visibilityLevel": event.get("visibilityLevel", ""),
                "date": date,
                "hasMoreDates": event.get("hasMoreDates", False),
                "onlineBookable": event.get("onlineBookable", False),
                "place": location.get("place", ""),
                "town": location.get("town", ""),
                "regions": ", ".join(location.get("regions", [])),
                "country": location.get("country", ""),
                "latitude": coordinate.get("lat"),
                "longitude": coordinate.get("long"),
                "description_short": event.get("plainDescriptions", [{}])[0].get("description", "") if event.get("plainDescriptions") else "",
                "description_long": long_description,
                "image_url": img_url,
                "urlFriendlyName": event.get("urlFriendlyName", ""),
                "start_times": start_times
            })
    return pd.DataFrame(rows)


if __name__ == "__main__":
    session_id = "G1758362087062"  # Replace with your real session ID
    events = fetch_unpli_events(page_no=1, page_size=100, session_id=session_id)

    if not events:
        print("No events or error fetching data.")
        exit(1)

    simplified_events = transform_events_for_json(events, session_id)

    with open("../dataset/veneto_unpliveneto_events.json", "w", encoding="utf-8") as f:
        json.dump({"events": simplified_events}, f, ensure_ascii=False, indent=4)


# per ora commento la visualizzazione perchè mi serve solo il json
#    df = events_to_dataframe(events, session_id)
#    try:
#        from IPython.display import display
#        display(df)
#    except ImportError:
#        print(df.to_string())
